# 4.文本与字节序列
## 4.1 字符问题
1. Unicode
    + 字符的标识-码位字符的标识，即码位，是0~1114111 的数字（十进制），在Unicode 标准中以4~6个十六进制数字表示，而且加前缀“U+”。例如，字母A的码位是U+0041，欧元符号的码位是U+20AC
    + 字符的具体表述取决于所用的编码，即utf-8、utf-16。在 UTF-8 编码中，A (U+0041）的码位编码成单个字节\x41，而在 UIF-16LE 编码中编码成两个字节\x41\x00。
2. 编码:encode，解码:decode

## 4.2 字节概要
1. bytes 与 bytearray对象的各个元素都是介于0~255之间的整数
2. 二进制序列的切片始终都是同一类型的二进制序列，包括长度为1的切片
3. [bytes VS bytearray](https://stackoverflow.com/questions/62903377/python3-bytes-vs-bytearray-and-converting-to-and-from-strings?newreg=4d248673031847c88f49f59faf8cb372)
    bytes 不可变、bytearray可变
4. 构建bytes bytearray 实例，传参：
    + 一个 str 对象和一个 encoding 关键字参数。
    + 一个可迭代对象，提供0~255之间的数值。
    + 一个实现了缓冲协议的对象（如bytes、 bytearray、 memoryview、 array.array)；此时，把源对象中的字节序列复制到新建的二进制序列中。——底层操作，可能涉及类型转换

In [20]:
cafe  = bytes('café', encoding='utf-8')
print(cafe)
print(cafe[0])#bytes对象的元素为整数
print(cafe[:1])#bytes的切片始终为bytes对象
print(cafe[0]==cafe[:1])
cafe_arr = bytearray(cafe)
print(cafe_arr)
print(cafe_arr[:1])#bytearray 对象的切片仍为bytearray
try:
    cafe[0] = 97
except Exception as e:
    print(e)

cafe_arr[0] = 97
print(cafe_arr)

b'caf\xc3\xa9'
99
b'c'
False
bytearray(b'caf\xc3\xa9')
bytearray(b'c')
'bytes' object does not support item assignment
bytearray(b'aaf\xc3\xa9')


In [21]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])#指定类型h
octets = bytes(numbers)
print(octets)

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'
